# Synthesis of results for Kerala landslide study

In [1]:
suppressMessages(library("rwwa"))

In [2]:
# function to fill infinite probability ratios
infer_infinite <- function(models_in) {
    
    # split out the columns for easier reference
    est <- est_n <- models_in[,1]
    l <- l_n <- models_in[,2]
    u <- u_n <- models_in[,3]
    
    # relabel where infinite best estimate / upper bound occurs
    mnames <- rownames(models_in)
    mnames <- paste0(mnames, c("*","")[is.finite(est)+1])
    mnames <- paste0(mnames, c("*","")[is.finite(u)+1])
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # INFINITE BEST ESTIMATES / UPPER BOUNDS
    # get upper bound where best estimate is infinite
    u_n[!is.finite(u_n)] <- (exp(log(est) + 3 * (log(est) - log(l))))[!is.finite(u_n)]

    # get best estimate, if infinite
    if(any(is.finite(u))) {
        # if any estimated upper bounds are finite, use the maximum upper bound to infer infinite best estimates
        f_upper <- max(u[is.finite(u)])
    } else {
        # otherwise, use the maximum inferred upper bound to infer infinite best estimates
        f_upper <- max(u_n[is.finite(u_n)])
    }
    est_n[!is.finite(est_n)] <- f_upper

    # repeat estimation of upper bound using inferred best estimate
    u_n[!is.finite(u_n)] <- (exp(log(est_n) + 3 * (log(est_n) - log(l))))[!is.finite(u_n)]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # INFINITE LOWER BOUNDS NOT YET IMPLEMENTED

    return(data.frame("est" = est_n, "lower" = l_n, "upper" = u_n, row.names = mnames))   
}

## Synthesis

In [3]:
varnm <- "rx1day"
st <- "rel"

obs_in <- read.csv(paste0("res-obs_",varnm,".csv"), row.names = "Dataset")
# obs_in <- obs_in[c("IMD (1979-2024)", "CPC"),]
obs_in <- obs_in[c("IMD (1901-2024)", "CPC"),]

models_in <- read.csv(paste0("res-models_",varnm,"-obsgmst.csv"), row.names = "Model")
models_in <- models_in[models_in$include == "y",]
rownames(models_in) <- gsub("rcp85_r1i1p1_","",rownames(models_in))

In [4]:
synth_dI_attr <- synthesis(obs_in[,grepl("dI", colnames(obs_in))],
                           models_in[,grepl("^di", colnames(models_in))],
                           synth_type = st)

synth_dI_proj <- synthesis(obs = NA, models_in[,grepl("fdi", colnames(models_in))], synth_type = st)

synth_PR_attr <- synthesis(infer_infinite(obs_in[,grepl("pr", colnames(obs_in))]),
                           infer_infinite(models_in[,grepl("^pr", colnames(models_in))]), synth_type = "PR")

synth_PR_proj <- synthesis(obs = NA, models_in[,grepl("fpr", colnames(models_in))], synth_type = "PR")

# save everything as .csv for easier plotting in Python
for(a in c("dI", "PR")) for (b in c("attr", "proj")) {
    s <- paste0("synth_",a,"_",b)
    write.csv(get(s)$df, paste0("synthesis/",s,"_",varnm,"-obsgmst-1901.csv"), row.names = F)
}

## Synthesis of projected changes excluding HighResMIP

In [5]:
# remove HighResMIP models
hrmip <- c('FGOALS-f3-L','FGOALS-f3-H','MRI-AGCM3-2-H','MRI-AGCM3-2-S','IPSL-CM6A-ATM-HR')
models_in <- models_in[!sapply(rownames(models_in), "%in%", hrmip),]

In [6]:
synth_dI_proj <- synthesis(obs = NA, models_in[,grepl("fdi", colnames(models_in))], synth_type = st)
synth_PR_proj <- synthesis(obs = NA, models_in[,grepl("fpr", colnames(models_in))], synth_type = "PR")


In [22]:
# save everything as .csv for easier plotting in Python
for(a in c("dI", "PR")) for (b in c("proj")) {
    s <- paste0("synth_",a,"_",b)
    write.csv(get(s)$df, paste0("synthesis/",s,"_",varnm,"-obsgmst-no-highresmip.csv"), row.names = F)
}

## Synthesis with IMD 1901-2024

In [8]:
varnm <- "rx1day"
st <- "rel"

obs_in <- read.csv(paste0("res-obs_",varnm,".csv"), row.names = "Dataset")
obs_in <- obs_in[c("IMD (1901-2024)", "CPC"),]

models_in <- read.csv(paste0("res-models_",varnm,"-obsgmst.csv"), row.names = "Model")
models_in <- models_in[models_in$include == "y",]
rownames(models_in) <- gsub("rcp85_r1i1p1_","",rownames(models_in))

In [9]:
synth_dI_attr <- synthesis(obs_in[,grepl("dI", colnames(obs_in))],
                           models_in[,grepl("^di", colnames(models_in))],
                           synth_type = st)

synth_PR_attr <- synthesis(infer_infinite(obs_in[,grepl("pr", colnames(obs_in))]),
                           infer_infinite(models_in[,grepl("^pr", colnames(models_in))]), synth_type = "PR")

# save everything as .csv for easier plotting in Python
for(a in c("dI", "PR")) for (b in c("attr")) {
    s <- paste0("synth_",a,"_",b)
    write.csv(get(s)$df, paste0("synthesis/",s,"_",varnm,"-obsgmst-1901.csv"), row.names = F)
}